<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import urllib.request
import nltk
nltk.download('punkt')

urls = ['https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-ABC%20%EC%82%B4%EC%9D%B8%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EA%B7%B8%EB%A6%AC%EA%B3%A0%20%EC%95%84%EB%AC%B4%EB%8F%84%20%EC%97%86%EC%97%88%EB%8B%A4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%82%98%EC%9D%BC%EA%B0%95%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A7%8C%EC%B0%AC%ED%9A%8C%EC%9D%98%2013%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A9%94%EC%86%8C%ED%8F%AC%ED%83%80%EB%AF%B8%EC%95%84%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%AA%A9%EC%82%AC%EA%B4%80%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B2%99%EC%96%B4%EB%A6%AC%20%EB%AA%A9%EA%B2%A9%EC%9E%90.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B9%84%EB%B0%80%20%EC%84%9C%EB%A5%98%EB%A5%BC%20%EB%85%B8%EB%A0%A4%EB%9D%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%8A%A4%ED%8E%98%EC%9D%B8%EA%B6%A4%EC%A7%9D%EC%9D%98%20%EB%B9%84%EB%B0%80.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%84%ED%8C%8C%ED%8A%B8%EC%97%90%20%EB%82%98%ED%83%80%EB%82%9C%20%EC%9A%94%EC%A0%95.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%A0%ED%81%AC%EB%A1%9C%EC%9D%B4%EB%93%9C%20%EC%82%B4%EC%9D%B8%20%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%98%88%EA%B3%A0%20%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%A5%90%EB%8D%AB.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%BB%A4%ED%8A%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%81%AC%EB%A6%AC%EC%8A%A4%EB%A7%88%EC%8A%A4%20%ED%91%B8%EB%94%A9%EC%9D%98%20%EB%AA%A8%ED%97%98.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%91%B8%EB%A5%B8%EC%97%B4%EC%B0%A8%EC%9D%98%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%99%94%EC%9A%94%EC%9D%BC%20%ED%81%B4%EB%9F%BD%EC%9D%98%20%EC%82%B4%EC%9D%B8.txt']

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_korean_sentences(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40:
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
korean_sentences = collect_korean_sentences(urls)

In [ ]:
len(korean_sentences)

18036

In [ ]:
korean_sentences[100]

'그러나 당신이 받으셨다는 그 이상한 편지도 고려해 보고 싶습니다, 포아로 씨.'

# 한글 뉴스 크롤러

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [95]:
f = open("/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv", "a")
f.write("title,contents\r\n")
f.close() 

In [92]:
import sys
import requests
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool

client_id = "waXxTjNdGjzj8xtT3nbF" #1.에서 취득한 아이디 넣기
client_secret = "H6mejnkuYx"  #1. 에서 취득한 키 넣기


# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def clean_html(x):
    x = re.sub("\&\w*\;","",x)
    x = re.sub("<.*?>","",x)
    return x

def get_news_urls(search_word = '코로나',max_display = 100):
    encode_type = 'json' #출력 방식 json 또는 xml
    #max_display = 100 #출력 뉴스 수
    sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
    start = 1 # 출력 위치

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #print(url)
    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    df = None
    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        #요청 결과 보기 200 이면 정상적으로 요청 완료
        df = pd.DataFrame(r.json()['items'])
        df['title'] = df['title'].apply(lambda x: clean_html(x))
        df['description'] = df['description'].apply(lambda x: clean_html(x))
    return df

def get_news_title_contents(url):
    headers = {'User-Agent':'Chrome/66.0.3359.181'}
    main_news = []
    result = requests.get(url,headers=headers)
    if result.status_code == 200:
        bs_obj = BeautifulSoup(result.content, "html.parser")
        #print(bs_obj)
        try:
            title = bs_obj.find("h3",{"id":"articleTitle"}).text
            body = bs_obj.find("div",{"class":"_article_body_contents"}).get_text("\n").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}","")
            #aid = bs_obj.find("div",{"class":"article_header"}).find("img")["title"]
            main_news = [clean_text(title),clean_text(body)]
        except:
            print("Unexpected error:", sys.exc_info()[0])
    return main_news    

In [93]:
def make_corpus(search_word = '코로나'):
    f = open("/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv", "a")
    df = get_news_urls(search_word = search_word,max_display = 100)
    if df is not None:
        for url in df['link']:
            if url.startswith('https://news.naver.com/'):
                #print(url)
                news = get_news_title_contents(url)
                if (len(news) == 2):
                    print('title:',news[0])
                    f.write(news[0] + "," + news[1] + "\r\n")
    f.close() 

In [94]:
keywords = ['뉴스','속보','스포츠','광명','금융','자동','인터뷰','대출','신도시','바이오','경쟁','공무원','재난','신도시','청약','주문','협력','교회','충성','요양','병원','백신']
keywords += ['택시','유족','방치','DNA','중대본','공수처','검찰','서울','시장','단일화','시간표','야권','협상','여론','중도','특검','포비아','구토','수도권','증강현실','게임','선물','시계','장사','거리','허탈','IPO','상견례','결정','악재','외국계','희소식','배터리','질주']
keywords += ['미세먼지','봄비','전국','소식','강원','우산','미나리','뉴욕','미국','출연','배우','아카네미','기생충','오스카','방탄','소년','BTS','앨범','판매','누리꾼','클릭','차트','음반','폭스바겐','아틀라스','유지','비용','세단','화사','궁궐','왕릉','유적','개화','절정','4월','문화재','조선','헤드','라인','국가','대표']
keywords += ['접종','중단']
keywords += ['유럽','국가','예방','선업','특정','태국','연기','혈전','관계','확인','차원','혈액','응고','총리','접종','취소','부작용','갈등','발화','홍콩','보고','입법부','인사','자리','선거','개편','배척','통과','투표','버튼','미소','시진핑']
len(keywords)

131

In [96]:
for word in keywords:
    print('keyword:',word)
    make_corpus(search_word = word)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
title: 대전 첫 코로나19 예방접종센터 이달 하순 개소…19일 모의 훈련
Unexpected error: <class 'AttributeError'>
title: 황경수 대한씨름협회장 거제시청-수원시청씨름단 합동훈련장 격려 방문
title: 대전시-종합병원 코로나19 백신접종 협약 체결
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
title: 충북 65세 이상 29만6000명 3월 말부터 AZ백신 접종
title: 김포시 한강야생조류생태공원서 찐 힐링 맛보자
title: 대전시·병원 코로나19 예방접종 인력지원협약 체결
Unexpected error: <class 'AttributeError'>
title: [SPECIAL] 열정과 땀 스포츠 정신으로 승부하다! 스포츠 기자
title: 지연된 정의? 어설픈 대응? 박철우 구타 이상열 KB감독 끝내 사퇴
keyword: 광명
title: 文 변창흠 책임지는 모습 보여야…사의 수용
title: 에이치에너지 전력O2O K-스타트업대상 수상
Unexpected error: <class 'AttributeError'>
title: LH 직원 대출 집중 북시흥농협 심사과정서 LH직원 신분 알아
title: 로열티 없는 갑각류찜 프랜차이즈 헬로크랩 오는 4월 3개 가맹점 오픈 예정
title: 與 서영석·김주영도 신도시 인근 땅 매입…투기 목적 아냐(종합)
title:

KeyboardInterrupt: ignored

In [97]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
3937,광주광역시 동구 ‘카카오톡 무료심리상담 사업’ 추진,비대면 카카오톡 무료심리상담 사업 안내 포스터. (제공광주광역시 동구) [광주스포츠...
3938,[마켓인사이트]교보 신창재 회장 안진-FI 여론 압박전 총력,≪이 기사는 03월12일(16:59) 자본시장의 혜안 ‘마켓인사이트’ 에 게재된 기...
3939,문체부 상대 OTT 저작권료 소송에 KT·LGU+도 참여,지난달 웨이브·티빙·왓챠 이어 소송 제기 (지디넷코리아김민선 기자)정부를 상대로 한...
3940,정부 갈등에도…美·中 반도체 기업 의기투합,20여개 기업 협의체 만들어 수출 제한·공급망 등 논의키로 중국반도체산업협회(CSI...
